<a href="https://colab.research.google.com/github/jeffheaton/present/blob/master/youtube/gan/colab_gan_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Apr  7 16:48:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Set Up New Environment

You will likely need to train for >24 hours.  Colab will disconnect you.  You must be prepared to restart training when this eventually happens.  Training is divided into ticks, every so many ticks (50 by default) your neural network is evaluated and a snapshot is saved.  When CoLab shuts down, all training after the last snapshot is lost. It might seem desirable to snapshot after each tick; however, this snapshotting process itself takes nearly an hour.  It is important to learn an optimal snapshot size for your resolution and training data.

We will mount GDRIVE so that your snapshots are saved there.  You must also place your training images in GDRIVE.

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


In [3]:
!python --version

Python 3.7.13


You must also install NVIDIA StyleGAN2 ADA PyTorch.  We also need to downgrade PyTorch to a version that supports StyleGAN.

In [43]:
!pip install numpy==1.16.4
!pip install tensorflow-gpu==1.14
#!pip install conda
#!conda install tensorflow-gpu==1.14
#!git clone https://github.com/NVlabs/stylegan2-ada
!git clone https://github.com/kokostino/stylegan2-ada
!pip install ninja

  Using cached tensorflow_estimator-1.14.0-py2.py3-none-any.whl (488 kB)
  Using cached tensorboard-1.14.0-py3-none-any.whl (3.1 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.5.3 which is incompatible.
tensorflow 1.15.2 requires tensorboard<1.16.0,>=1.15.0, but you have tensorboard 1.14.0 which is incompatible.
tensorflow 1.15.2 requires tensorflow-estimator==1.15.1, but you have tensorflow-estimator 1.14.0 which is i

fatal: destination path 'stylegan2-ada' already exists and is not an empty directory.


# Find Your Files

The drive is mounted to the following location.

```
/content/drive/MyDrive/data
```

It might be helpful to use an ```ls``` command to establish the exact path for your images.

In [ ]:
#!ls /content/drive/MyDrive/data/gan/images

# Convert Your Images

In [7]:
%tensorflow_version 1.x
import tensorflow as tf
#import os
#os.getcwd()
#sess = tf.Session()

TensorFlow 1.x selected.


In [ ]:
!python /content/stylegan2-ada/dataset_tool.py create_from_images /content/drive/MyDrive/data/gan/dataset /content/drive/MyDrive/data/gan/images

Loading images from "/content/drive/MyDrive/data/gan/images"
Creating dataset "/content/drive/MyDrive/data/gan/dataset"
Added 2280 images.


In [ ]:
!python /content/stylegan2-ada/dataset_tool.py display /content/drive/MyDrive/data/gan/dataset

Loading dataset "/content/drive/MyDrive/data/gan/dataset"
Displaying images
: cannot connect to X server 


The following command can be used to clear out the newly created dataset.  If something goes wrong and you need to clean up your images and rerun the above command, you should delete your partially created dataset directory.

In [ ]:
!rm -R /content/drive/MyDrive/data/gan/dataset/*

# Clean Up your Images

It is important that all images have the same dimensions and color depth.  This code can identify images that have issues.

In [ ]:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = '/content/drive/MyDrive/data/gan/images'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz


  0%|          | 0/2281 [00:00<?, ?it/s]

Inconsistant color format: /content/drive/MyDrive/data/gan/images/Screenshot_20210627-215429.png


# Perform Initial Training

In [ ]:
!nvcc /content/stylegan2-ada/test_nvcc.cu -o /content/stylegan2-ada/test_nvcc -run

CPU says hello.
GPU says hello.


In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
DATA = "/content/drive/MyDrive/data/gan/dataset"
SNAP = 25
LOSS = 'wgangp'

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA} --cmethod = {LOSS}"
!{cmd}


Training options:
{
  "G_args": {
    "func_name": "training.networks.G_main",
    "fmap_base": 8192,
    "fmap_max": 512,
    "mapping_layers": 2,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "D_args": {
    "func_name": "training.networks.D_main",
    "mbstd_group_size": 4,
    "fmap_base": 8192,
    "fmap_max": 512,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "G_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "D_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "loss_args": {
    "func_name": "training.loss.stylegan2",
    "r1_gamma": 0.8192
  },
  "augment_args": {
    "class_name": "training.augment.AdaptiveAugment",
    "tune_heuristic": "rt",
    "tune_target": 0.6,
    "apply_func": "training.augment.augment_pipeline",
    "apply_args": {
      "xflip": 1,
      "rotate90": 1,
      "xint": 1,
      "scale": 1,
      "rotate": 1,
      "aniso": 1,
      "xfrac": 1,
      "brightness": 1,
   

In [ ]:
os.getcwd()

'/content'

In [ ]:
!/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap 5 --resume /content/drive/MyDrive/data/gan/experiments/00000-dataset-auto1/network-snapshot-000020.pkl --outdir /content/drive/MyDrive/data/gan/experiments --data /content/drive/MyDrive/data/gan/dataset

Usage: train.py [OPTIONS]
Try 'train.py --help' for help.

Error: --data: Path must point to a directory or zip


# Resume Training

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
NETWORK = "network-snapshot-000028.pkl"
RESUME = os.path.join(EXPERIMENTS, "00003-dataset-auto1-resumecustom", NETWORK)
DATA = "/content/drive/MyDrive/data/gan/dataset"
SNAP = 7
LOSS = 'wgangp'

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA} --cmethod {LOSS} --aug=noaug --resume {RESUME}"
!{cmd}

Traceback (most recent call last):
  File "/content/stylegan2-ada/train.py", line 16, in <module>
    import tensorflow as tf
  File "/tensorflow-1.15.2/python3.7/tensorflow/__init__.py", line 99, in <module>
    from tensorflow_core import *
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/__init__.py", line 28, in <module>
    from tensorflow.python import pywrap_tensorflow  # pylint: disable=unused-import
  File "<frozen importlib._bootstrap>", line 1019, in _handle_fromlist
  File "/tensorflow-1.15.2/python3.7/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/tensorflow-1.15.2/python3.7/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/lib/python3.7/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/__init__.py", line 73, in <module>
    from tensorflow.python.ops.

# Resume with stylegan loss

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
NETWORK = "network-snapshot-000483.pkl"
RESUME = os.path.join(EXPERIMENTS, "00016-dataset-auto1-resumecustom", NETWORK)
DATA = "/content/drive/MyDrive/data/gan/dataset"
SNAP = 15
LOSS = 'wgangp'

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA} --resume {RESUME}"
!{cmd}


Training options:
{
  "G_args": {
    "func_name": "training.networks.G_main",
    "fmap_base": 8192,
    "fmap_max": 512,
    "mapping_layers": 2,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "D_args": {
    "func_name": "training.networks.D_main",
    "mbstd_group_size": 4,
    "fmap_base": 8192,
    "fmap_max": 512,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "G_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "D_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "loss_args": {
    "func_name": "training.loss.stylegan2",
    "r1_gamma": 0.8192
  },
  "augment_args": {
    "class_name": "training.augment.AdaptiveAugment",
    "tune_heuristic": "rt",
    "tune_target": 0.6,
    "apply_func": "training.augment.augment_pipeline",
    "apply_args": {
      "xflip": 1,
      "rotate90": 1,
      "xint": 1,
      "scale": 1,
      "rotate": 1,
      "aniso": 1,
      "xfrac": 1,
      "brightness": 1,
   

# Projecting

In [5]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
NETWORK = "network-snapshot-000362.pkl"
RESUME = os.path.join(EXPERIMENTS, "00015-dataset-auto1-resumecustom", NETWORK)
RESUME

'/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl'

In [16]:
!python /content/stylegan2-ada/projector.py --outdir=drive/MyDrive/data/gan/projections --save-video false --target=/content/drive/MyDrive/data/gan/mosque.png --network=/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Projector: Computing W midpoint and stddev using 10000 samples...
Projector: std = 18.6189
Projector: Setting up noise inputs...
Projector: Building image output graph...
Projector: Building loss graph...
Projector: Building noise regularization graph...
Projector: Setting up optimizer...
Projector: Preparing target images...
Projector: Initializing optimization state...
100% 1000/1000 [01:38<00:00, 10.13it/s, dist=0.2768, loss=0.28]


In [15]:
!python /content/stylegan2-ada/projector.py --outdir=drive/MyDrive/data/gan/projections --save-video false --target=/content/drive/MyDrive/data/gan/ams.png --network=/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Projector: Computing W midpoint and stddev using 10000 samples...
Projector: std = 18.6189
Projector: Setting up noise inputs...
Projector: Building image output graph...
Projector: Building loss graph...
Projector: Building noise regularization graph...
Projector: Setting up optimizer...
Projector: Preparing target images...
Projector: Initializing optimization state...
100% 1000/1000 [01:39<00:00, 10.03it/s, dist=0.4116, loss=0.41]


In [97]:
!python /content/stylegan2-ada/projector.py --outdir=drive/MyDrive/data/gan/projections --save-video false --target=/content/drive/MyDrive/data/gan/wmkk.png --network=/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Projector: Computing W midpoint and stddev using 10000 samples...
Projector: std = 18.6189
Projector: Setting up noise inputs...
Projector: Building image output graph...
Projector: Building loss graph...
Projector: Building noise regularization graph...
Projector: Setting up optimizer...
Projector: Preparing target images...
Projector: Initializing optimization state...
100% 1000/1000 [01:39<00:00, 10.02it/s, dist=0.2544, loss=0.25]


In [17]:
!python /content/stylegan2-ada/projector.py --outdir=drive/MyDrive/data/gan/projections --save-video false --target=/content/drive/MyDrive/data/gan/eastvan.png --network=/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Projector: Computing W midpoint and stddev using 10000 samples...
Projector: std = 18.6189
Projector: Setting up noise inputs...
Projector: Building image output graph...
Projector: Building loss graph...
Projector: Building noise regularization graph...
Projector: Setting up optimizer...
Projector: Preparing target images...
Projector: Initializing optimization state...
100% 1000/1000 [01:38<00:00, 10.19it/s, dist=0.3421, loss=0.34]


In [ ]:
!python /content/stylegan2-ada/style_mixing.py --rows=25,1600,75,358,1900 --cols=45,721,1189,293 --outdir=out_mix --network=/content/drive/MyDrive/data/gan/experiments/00011-dataset-auto1-resumecustom/network-snapshot-000362.pkl

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00011-dataset-auto1-resumecustom/network-snapshot-000362.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating W vectors...
Generating images...
Generating style-mixed images...
Saving images...
Saving image grid...


In [ ]:
!python /content/stylegan2-ada/style_mixing.py --rows=25,1600,75,358,1900 --cols=45,721,1189,293 --outdir=out_mix --network=/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl

In [ ]:
!python /content/stylegan2-ada/style_mixing.py --rows=85,100,75,458,1500 --cols=55,821,1789,293 --outdir=out_mix --network=/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl

In [ ]:
!python /content/stylegan2-ada/generate.py --outdir=out2 --dlatents="/content/drive/MyDrive/data/gan/dlatentsams.npz" --network=/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00011-dataset-auto1-resumecustom/network-snapshot-000362.pkl"...
Generating images from dlatents file "/content/drive/MyDrive/data/gan/dlatentsams.npz"
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Saved out2/dlatent00.png


#Project between latent vectors

1. Transform latent vectors to right shape

vec1 = usable_latent_thing("/content/drive/MyDrive/data/gan/projections/dlatentsvec1.npz")

vec2 = usable_latent_thing("/content/drive/MyDrive/data/gan/projections/dlatentsvec2.npz")


2. Find the interpolations between the latent vectors (default step size 10)

interpolated_vec = interpolate_points(vec1, vec2)

3. Save interpolated vectors

np.save("/content/drive/MyDrive/data/gan/projections/vec1_vec2_ratio", interpolated_vec)

4. Generate images from interpolated vectors

!python /content/stylegan2-ada/generate.py --outdir=drive/MyDrive/data/gan/mix --dlatents="/content/drive/MyDrive/data/gan/projections/vec1_vec2_ratio.npy" --network=/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl

In [18]:
# Include in generator.py

import numpy as np
def usable_latent_thing(dir):
  dl=np.load(dir)['dlatents']
  a=dl[0][0]
  b=np.array([[a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a]])
  return b
def usable_latent_thing_simple(dir):
  dl=np.load(dir)
  return dl

#try:
#      dlatents = usable_latent_thing(dlatents_npz)
#except:
#      dlatents = usable_latent_thing_simple(dlatents_npz)
#assert dlatents.shape[1:] == (18, 512) # [N, 18, 512]

In [94]:
np.load("/content/drive/MyDrive/data/gan/projections/ams_mos_ratio.npy")[0].shape

(18, 512)

In [27]:
ams=usable_latent_thing("/content/drive/MyDrive/data/gan/projections/dlatentsams.npz")
mos=usable_latent_thing("/content/drive/MyDrive/data/gan/projections/dlatentsmos.npz")

In [41]:
np.save("/content/drive/MyDrive/data/gan/projections/ams", ams)

In [44]:
mix=(ams+mos)/2
np.save("/content/drive/MyDrive/data/gan/projections/mix", mix)

In [46]:
ams

array([[[-2.1739964 ,  0.02426555, -0.8351016 , ..., -0.823681  ,
         -3.5707688 ,  0.2852919 ],
        [-2.1739964 ,  0.02426555, -0.8351016 , ..., -0.823681  ,
         -3.5707688 ,  0.2852919 ],
        [-2.1739964 ,  0.02426555, -0.8351016 , ..., -0.823681  ,
         -3.5707688 ,  0.2852919 ],
        ...,
        [-2.1739964 ,  0.02426555, -0.8351016 , ..., -0.823681  ,
         -3.5707688 ,  0.2852919 ],
        [-2.1739964 ,  0.02426555, -0.8351016 , ..., -0.823681  ,
         -3.5707688 ,  0.2852919 ],
        [-2.1739964 ,  0.02426555, -0.8351016 , ..., -0.823681  ,
         -3.5707688 ,  0.2852919 ]]], dtype=float32)

In [34]:
!python /content/stylegan2-ada/generate.py --outdir=drive/MyDrive/data/gan/mix --dlatents="/content/drive/MyDrive/data/gan/projections/dlatentsams.npz" --network=/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl"...
Generating images from dlatents file "/content/drive/MyDrive/data/gan/projections/dlatentsams.npz"
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Saved drive/MyDrive/data/gan/dlatent00.png


In [49]:
!python /content/stylegan2-ada/generate.py --outdir=drive/MyDrive/data/gan/mix --dlatents="/content/drive/MyDrive/data/gan/projections/mix.npy" --network=/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl"...
Generating images from dlatents file "/content/drive/MyDrive/data/gan/projections/mix.npy"
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Saved drive/MyDrive/data/gan/mix/dlatent00.png


In [86]:
ams[0].shape

(18, 512)

In [87]:
# uniform interpolation between two points in latent space
def interpolate_points(p1, p2, n_steps=10):
	# interpolate ratios between the points
	ratios = np.linspace(0, 1, num=n_steps)
	# linear interpolate vectors
	vectors = list()
	for ratio in ratios:
		v = (1.0 - ratio) * p1[0] + ratio * p2[0]
		vectors.append(v)
	return np.asarray(vectors)

In [88]:
interpolated_vec = interpolate_points(ams,mos)
interpolated_vec.shape

(10, 18, 512)

In [89]:
np.save("/content/drive/MyDrive/data/gan/projections/ams_mos_ratio", interpolated_vec)

In [95]:
!python /content/stylegan2-ada/generate.py --outdir=drive/MyDrive/data/gan/mix --dlatents="/content/drive/MyDrive/data/gan/projections/ams_mos_ratio.npy" --network=/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00015-dataset-auto1-resumecustom/network-snapshot-000362.pkl"...
Generating images from dlatents file "/content/drive/MyDrive/data/gan/projections/ams_mos_ratio.npy"
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Saved drive/MyDrive/data/gan/mix/dlatent00.png
Saved drive/MyDrive/data/gan/mix/dlatent01.png
Saved drive/MyDrive/data/gan/mix/dlatent02.png
Saved drive/MyDrive/data/gan/mix/dlatent03.png
Saved drive/MyDrive/data/gan/mix/dlatent04.png
Saved drive/MyDrive/data/gan/mix/dlatent05.png
Saved drive/MyDrive/data/gan/mix/dlatent06.png
Saved drive/MyDrive/data/gan/mix/dlatent07.png
Saved drive/MyDrive/data/gan/mix/dlatent08.png
Saved drive/MyDrive/data/gan/mix/dlatent09.png


generate.py mit


def usable_latent_thing(dir):
  dl=np.load(dir)['dlatents']
  a=dl[0][0]
  b=np.array([[a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a]])
  return b

def usable_latent_thing_simple(dir):
  dl=np.load(dir)
  a=dl[0][0]
  b=np.array([[a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a]])
  return dl

def generate_images(network_pkl, seeds, truncation_psi, outdir, class_idx, dlatents_npz):
    tflib.init_tf()
    print('Loading networks from "%s"...' % network_pkl)
    with dnnlib.util.open_url(network_pkl) as fp:
        _G, _D, Gs = pickle.load(fp)

    os.makedirs(outdir, exist_ok=True)

    # Render images for a given dlatent vector.
    if dlatents_npz is not None:
        print(f'Generating images from dlatents file "{dlatents_npz}"')
        try:
          dlatents = usable_latent_thing(dlatents_npz)
        except:
          dlatents = usable_latent_thing_simple(dlatents_npz)
        #assert dlatents.shape[1:] == (18, 512) # [N, 18, 512]
        imgs = Gs.components.synthesis.run(dlatents, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
        for i, img in enumerate(imgs):
            fname = f'{outdir}/dlatent{i:02d}.png'
            print (f'Saved {fname}')
            PIL.Image.fromarray(img, 'RGB').save(fname)
        return